In [ ]:
from qiskit.tools.jupyter import *
from qiskit import pulse
from qiskit.circuit import QuantumCircuit, Gate, Parameter, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Measure
from qiskit.pulse import Acquire, AcquireChannel, MemorySlot
from qiskit import schedule, transpile, execute

import numpy as np
import matplotlib.pyplot as plt 

## Connect to IBM backend with Pulse

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibm_kyoto') 

In [ ]:
backend_config = backend.configuration()
dt = backend_config.to_dict()['dtm']
assert backend_config.open_pulse, "Backend doesn't support Pulse"

## Define readout pulse sequence 

In [ ]:
qubit = 0
duration = 560 # set desired duration in units of dt, must be multiple of 8

with pulse.build(backend=backend, name='measure_sched') as measure_sched:
    meas_chan = pulse.measure_channel(qubit)
    acq_chan = pulse.acquire_channel(qubit)
    with pulse.align_left():
        # amplitude and phase obtained by inspecting the pulse parameters for qc.measure(0)$
        pulse.play(pulse.Constant(duration=duration, amp=0.36, angle=-0.214), meas_chan)
        pulse.acquire(duration, acq_chan, MemorySlot(0))

measure_gate = Gate("measure_gate", 1, [duration])

qc_meas_0 = QuantumCircuit(1, 1)
qc_meas_0.append(measure_gate, [0]) 
qc_meas_0.add_calibration(measure_gate, (0,), measure_sched)

qc_meas_1 = QuantumCircuit(1, 1)
qc_meas_1.x(0)
qc_meas_1.append(measure_gate, [0]) 
qc_meas_1.add_calibration(measure_gate, (0,), measure_sched)

all_circs = [qc_meas_0, qc_meas_1]

In [ ]:
sweep_schedule = schedule(all_circs, backend)
sweep_schedule[1].draw(backend=backend)

## Run experiment

In [ ]:
job = backend.run(all_circs, meas_level=1, meas_return='single', shots=2048)

## Load data 

In [ ]:
fns = [
    'duration_560_kyoto_4910_7160.pkl',
    'duration_1096_kyoto_4910_7160.pkl',
    'duration_2496_kyoto_4910_7160.pkl',
    'duration_4992_kyoto_4910_7160.pkl'
] 

results = []
for fn in fns:
    with open('data/' + fn, 'rb') as f:
        results.append(pickle.load(f))

## Analysis

In [ ]:
from scipy.optimize import curve_fit

def gaussian(x, mu, sigma, A):
    return A * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

# Double Gaussian function
def double_gaussian(x, mu1, sigma1, A1, mu2, sigma2, A2):
    return A1 * np.exp(-0.5 * ((x - mu1) / sigma1) ** 2) + A2 * np.exp(-0.5 * ((x - mu2) / sigma2) ** 2)


def intersection_of_two_gaussians(mu1, sigma1, A1, mu2, sigma2, A2):
    a = -0.5*(1/sigma2**2 - 1/sigma1**2)
    b = (mu2/sigma2**2 - mu1/sigma1**2)
    c = np.log(A2/A1) - 0.5*((mu2/sigma2)**2 - (mu1/sigma1)**2)
    x1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    x2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    return x1, x2

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), nrows=2, ncols=2, sharex=True, sharey=True)

for i, result in enumerate(results):
    IQ_0 = result.get_memory(0)
    IQ_1 = result.get_memory(1)
    # plotting indices
    xi, xj = i//2, i%2
    ax[xi,xj].plot(np.real(IQ_0), np.imag(IQ_0), 'o', ms=2, alpha=0.2)
    ax[xi,xj].plot(np.real(IQ_1), np.imag(IQ_1), 'o', ms=2, alpha=0.2)
    ax[xi,xj].grid()
    
ax[0,0].set_ylabel('Q')
ax[1,0].set_ylabel('Q')
ax[1,0].set_xlabel('I')
ax[1,1].set_xlabel('I')

fig.suptitle('Raw Qiskit integrated readout IQ data')
fig.tight_layout()

In [ ]:
dt_to_ns = 0.5

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(7,7), sharex=True, sharey=True)

durations = [560, 1096, 2496, 4992]

misprep_eg_list = []
misprep_ge_list = []

for i, result in enumerate(results):
    
    # plotting indices
    xi, xj = i//2, i%2
    
    data_0 = result.get_memory(0)
    data_1 = result.get_memory(1)
        
    # bin experimental data
    counts_0, bins_0 = np.histogram(np.real(data_0), bins=40)
    counts_1, bins_1 = np.histogram(np.real(data_1), bins=40)
    bin_centers_0 = (bins_0[:-1] + bins_0[1:])/2
    bin_centers_1 = (bins_1[:-1] + bins_1[1:])/2
    
    # plot binned experimental data
    ax[xi,xj].plot(bin_centers_0, counts_0, 'o', ms=3, c='C0', label=None if i > 0 else 'prep_g')
    ax[xi,xj].plot(bin_centers_1, counts_1, 'o', ms=3, c='C1', label=None if i > 0 else 'prep_e')
    ax[xi,xj].set_yscale('log')
    ax[xi,xj].set_ylim(10**0, 10**2.5)
    
    # fit double gaussians
    p0_0 = [-0.1*(i+1), 0.3, np.max(counts_0), 0.1*(i+1), 0.3, np.max(counts_0)/5]
    p0_1 = [-0.1*(i+1), 0.3, np.max(counts_0)/5, 0.1*(i+1), 0.3, np.max(counts_0)]
    params_0, cov = curve_fit(double_gaussian, bin_centers_0/1e8, counts_0, p0=p0_0, maxfev=10000)
    params_1, cov = curve_fit(double_gaussian, bin_centers_1/1e8, counts_1, p0=p0_1, maxfev=10000)
    
    # determine bounds for plotting fits
    xmin = np.min([bins_0, bins_1])
    xmax = np.max([bins_0, bins_1])
    fit_xs = np.linspace(xmin, xmax, 100)
    
    # calculate decision boundary
    threshold_x = 1e8 * intersection_of_two_gaussians(*params_0[:3], *params_1[3:])[0]
    
    # calculate fidelity F = 1 - P(g|e) - P(e|g)
    N = data_0.shape[0]
    P_eg = np.sum(np.where(np.real(data_0) > threshold_x, 1, 0)) / N
    P_ge = np.sum(np.where(np.real(data_1) < threshold_x, 1, 0)) / N
    F = 1 - P_ge - P_eg
    gaussian_g = gaussian(fit_xs/1e8, *params_0[:3])
    gaussian_e = gaussian(fit_xs/1e8, *params_1[3:])
    P_eg_ideal = np.sum(gaussian_g[np.argwhere(fit_xs > threshold_x)]) / np.sum(gaussian_g)
    P_ge_ideal = np.sum(gaussian_e[np.argwhere(fit_xs < threshold_x)]) / np.sum(gaussian_e)
    F_ideal = 1 - P_ge_ideal - P_eg_ideal
    
    misprep_eg = params_0[5]/(params_0[2] + params_0[5]) # expect g get e
    misprep_ge = params_1[2]/(params_1[2] + params_1[5]) # expect e get g
    misprep_eg_list.append(misprep_eg)
    misprep_ge_list.append(misprep_ge)
    
    ax[xi,xj].plot(fit_xs, double_gaussian(fit_xs/1e8, *params_0), c='C0')
    ax[xi,xj].plot(fit_xs, gaussian(fit_xs/1e8, *params_0[:3]), '--', c='C0')
    ax[xi,xj].plot(fit_xs, double_gaussian(fit_xs/1e8, *params_1), c='C1')
    ax[xi,xj].plot(fit_xs, gaussian(fit_xs/1e8, *params_1[3:]), '--', c='C1')
    ax[xi,xj].fill_between(
        fit_xs, 0, np.minimum(gaussian(fit_xs/1e8, *params_1[3:]), gaussian(fit_xs/1e8, *params_0[:3])),
        alpha=0.4, color='C2', label=None if i!=0 else r'$\propto P(g|e)+P(e|g)$')
    ax[xi, xj].vlines(threshold_x, ymin=10**0, ymax=10**2.5, color='gray', linestyle='dashed', label=None if i>0 else 'threshold')
    
    ax[xi,xj].set_title(r"$\tau=$" + f"{durations[i]*0.5:.0f} ns, "  \
        + r'$F=$'+f'{F:.2f}, ' + r'$F_\mathrm{ideal}=$' + f'{F_ideal:.2f}')
    

ax[1,0].set_xlabel('I quadrature')
ax[1,1].set_xlabel('I quadrature')
ax[0,0].set_ylabel('Counts')
ax[1,0].set_ylabel('Counts')

fig.suptitle('Readout I quadrature distribution at varying measurement times \n with double Gaussian fits, n_shots=2048')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.93), ncol=4)
fig.tight_layout(rect=[0, 0, 1, 0.95])

In [ ]:
fig, ax = plt.subplots()

ax.plot(np.array(durations)*0.5, np.abs(misprep_eg_list), 'o-', label='P(e|g)')
ax.plot(np.array(durations)*0.5, np.abs(misprep_ge_list), 'o-', label='P(g|e)')
ax.set_ylabel('Probability of wrong state')
ax.set_xlabel(r'Readout duration $\tau$ [ns]')
ax.set_title('State "preparation" error vs readout duration')
ax.legend()